In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import aiohttp
import tqdm
import keepa
import os
import asyncio

#plt.style.use('whitegrid') # Matplotlib style
plt.rcParams['figure.figsize'] = [6, 3] # Matplotlib figure size


In [52]:
sep_dfA = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin A/Keepa Vitamin A Sep23.csv')
sep_dfA['Month'] = 'September'

# Load additional CSV files
oct_dfA = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin A/Keepa Vitamin A Oct23.csv')
oct_dfA = oct_dfA[oct_dfA['ASIN'].isin(sep_dfA['ASIN'])]
oct_dfA['Month'] = 'October'

nov_dfA = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin A/Keepa Vitamin A Nov23.csv')
nov_dfA = nov_dfA[nov_dfA['ASIN'].isin(sep_dfA['ASIN'])]
nov_dfA['Month'] = 'November'

dec_dfA = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin A/Keepa Vitamin A Dec23.csv')
dec_dfA = dec_dfA[dec_dfA['ASIN'].isin(sep_dfA['ASIN'])]
dec_dfA['Month'] = 'December'

jan_dfA = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin A/Keepa Vitamin A Jan24.csv')
jan_dfA = jan_dfA[jan_dfA['ASIN'].isin(sep_dfA['ASIN'])]
jan_dfA['Month'] = 'January'

feb_dfA = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin A/Keepa Vitamin A Feb24.csv')
feb_dfA = feb_dfA[feb_dfA['ASIN'].isin(sep_dfA['ASIN'])]
feb_dfA['Month'] = 'February'
final_dfA = pd.concat([sep_dfA, oct_dfA, nov_dfA, dec_dfA, jan_dfA, feb_dfA], ignore_index=True)

def parse_sales_rank(string):
    """
    'Sales Rank: Subcategory Sales Ranks'
        "# 3 | Top 0.03% | Vitamin A Supplements"
    Split this into three:
        [3, 0.03, 'Vitamin A Supplements']
        
    Sometimes input is like
        "# 3 | Top 0.03% | Vitamin A Supplements; # 13 | Top 0.13% | Candy"
    In this case, split by ; and take the string that contains 'Vitamin A Supplements'.
    """
    # select the category that contains 'Vitamin C Supplements'
    try:
        categories = string.split(';')
    except:
        # return None if the input is not a string.
        return pd.Series([None, None, None])
    Vitamin_A = [a for a in categories if a.find('Vitamin A Supplements') > -1]
    Vitamin_A = Vitamin_A[0] if len(Vitamin_A) > 0 else None

    try:
        rank = re.findall(r'#\s(\d+)\s\|', Vitamin_A.replace(',',''))
        rank = int(rank[0].strip())
    except:
        # print('rank \n', string) # print it to see what's wrong
        rank = None  # there is Calcium Mineral Supplements which is not Vitamin C
    
    # percent
    # 1. Find the string that starts with 'Top ' and ends with '%'.
    # 2. Return the number as a float.
    # 3. If the input is not a string, return None.
    try:
        percent = re.findall(r'Top\s(.*)%', Vitamin_A)
        percent = float(percent[0].strip())
    except:
        percent = None  # percent info might not exist if ranking is too high.
        # print('percent \n', string) # print it to see what's wrong
    
    # Category
    # 1. Split the string by '|' and take the last element.
    try:
        category = Vitamin_A.split('|')[-1].strip()
    except:
        # print('category \n', string)
        category = None # there is Calcium Mineral Supplements which is not Vitamin C
    return pd.Series([rank, percent, category])
    
final_dfA[['Absolute Rank', 'Top 1 Percent', 'Category']] = final_dfA['Sales Rank: Subcategory Sales Ranks'].apply(parse_sales_rank)    

# remove unnecessary columns
columns = ['Sales Rank: Subcategory Sales Ranks', 'Sales Rank: Reference', 'Top 1 Percent']
final_dfA = final_dfA.drop(columns, axis=1)

final_dfA.head()

,Locale,Image,Title,Sales Rank: Historical,Sales Rank: Current,Sales Rank: 90 days avg.,Sales Rank: Drops last 90 days,Bought in past month,Reviews: Rating,Reviews: Review Count,...,Adult Product,Trade-In Eligible,Prime Eligible (Buy Box),Subscribe and Save,One Time Coupon: Absolute,One Time Coupon: Percentage,Subscribe and Save Coupon: Percentage,Month,Absolute Rank,Category
0,com,https://m.media-amazon.com/images/I/711R2D6pCK...,"Bronson Vitamin A 10,000 IU Premium Non-GMO Fo...",2319.0,2210.0,2310.0,281.0,9000.0,4.7,18715.0,...,no,no,yes,yes,-,-,5 %,September,None,Vitamin A Supplements
1,com,https://m.media-amazon.com/images/I/511ijsPtft...,"NOW Supplements, Vitamin A 10,000 IU, Eye Heal...",5542.0,4192.0,5448.0,243.0,6000.0,4.7,1366.0,...,no,no,yes,yes,-,-,5 %,September,None,Vitamin A Supplements
2,com,https://m.media-amazon.com/images/I/71+A1pNv-A...,"Pure Encapsulations Vitamin A 10,000 IU from C...",8596.0,9164.0,8136.0,258.0,3000.0,4.7,2893.0,...,no,no,yes,no,-,-,-,September,None,Vitamin A Supplements
3,com,https://m.media-amazon.com/images/I/41632FER8z...,"Solgar Dry Vitamin A 1500 mcg (5000 IU), 100 T...",9348.0,35130.0,19535.0,165.0,500.0,4.7,2852.0,...,no,no,no,no,-,-,-,September,None,Vitamin A Supplements
4,com,https://m.media-amazon.com/images/I/41z3slcaYq...,"NOW Supplements, Vitamin A (Fish Liver Oil) 25...",10476.0,9701.0,8182.0,231.0,1000.0,4.7,7371.0,...,no,no,yes,yes,-,-,5 %,September,None,Vitamin A Supplements


In [57]:
sep_dfB = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin B/Keepa Vitamin B Sep23.csv')
sep_dfB['Month'] = 'September'

# Load additional CSV files
oct_dfB = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin B/Keepa Vitamin B Oct23.csv')
oct_dfB = oct_dfB[oct_dfB['ASIN'].isin(sep_dfB['ASIN'])]
oct_dfB['Month'] = 'October'

nov_dfB = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin B/Keepa Vitamin B Nov23.csv')
nov_dfB = nov_dfB[nov_dfB['ASIN'].isin(sep_dfB['ASIN'])]
nov_dfB['Month'] = 'November'

dec_dfB = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin B/Keepa Vitamin B Dec23.csv')
dec_dfB = dec_dfB[dec_dfB['ASIN'].isin(sep_dfB['ASIN'])]
dec_dfB['Month'] = 'December'

jan_dfB = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin B/Keepa Vitamin B Jan24.csv')
jan_dfB = jan_dfB[jan_dfB['ASIN'].isin(sep_dfB['ASIN'])]
jan_dfB['Month'] = 'January'

feb_dfB = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin B/Keepa Vitamin B Feb24.csv')
feb_dfB = feb_dfB[feb_dfB['ASIN'].isin(sep_dfB['ASIN'])]
feb_dfB['Month'] = 'February'
final_dfB = pd.concat([sep_dfB, oct_dfB, nov_dfB, dec_dfB, jan_dfB, feb_dfB], ignore_index=True)
def parse_sales_rank(string):
    """
    'Sales Rank: Subcategory Sales Ranks'
        "# 3 | Top 0.03% | Vitamin B Supplements"
    Split this into three:
        [3, 0.03, 'Vitamin B Supplements']
        
    Sometimes input is like
        "# 3 | Top 0.03% | Vitamin B Supplements; # 13 | Top 0.13% | Candy"
    In this case, split by ; and take the string that contains 'Vitamin B Supplements'.
    """
    # select the category that contains 'Vitamin B Supplements'
    try:
        categories = string.split(';')
    except:
        # return None if the input is not a string.
        return pd.Series([None, None, None])
    Vitamin_B = [b for b in categories if b.find('Vitamin B Supplements') > -1]
    Vitamin_B = Vitamin_B[0] if len(Vitamin_B) > 0 else None

    try:
        rank = re.findall(r'#\s(\d+)\s\|', Vitamin_B.replace(',',''))
        rank = int(rank[0].strip())
    except:
        # print('rank \n', string) # print it to see what's wrong
        rank = None  # there is Calcium Mineral Supplements which is not Vitamin C
    
    # percent
    # 1. Find the string that starts with 'Top ' and ends with '%'.
    # 2. Return the number as a float.
    # 3. If the input is not a string, return None.
    try:
        percent = re.findall(r'Top\s(.*)%', Vitamin_B)
        percent = float(percent[0].strip())
    except:
        percent = None  # percent info might not exist if ranking is too high.
        # print('percent \n', string) # print it to see what's wrong
    
    # Category
    # 1. Split the string by '|' and take the last element.
    try:
        category = Vitamin_B.split('|')[-1].strip()
    except:
        # print('category \n', string)
        category = None # there is Calcium Mineral Supplements which is not Vitamin C
    return pd.Series([rank, percent, category])

# Define the list of columns to drop again to avoid any previous overwrite or modification
columns_to_drop = ['Sales Rank: Subcategory Sales Ranks', 'Sales Rank: Reference', 'Top 1 Percent']
# Using the correctly defined list of columns to drop
final_dfB = final_dfB.drop(columns_to_drop, axis=1, errors='ignore')
final_dfB.head()


,Locale,Image,Title,Sales Rank: Historical,Sales Rank: Current,Sales Rank: 90 days avg.,Sales Rank: Drops last 90 days,Bought in past month,Reviews: Rating,Reviews: Review Count,...,Item: Weight (g),Hazardous Materials,Adult Product,Trade-In Eligible,Prime Eligible (Buy Box),Subscribe and Save,One Time Coupon: Absolute,One Time Coupon: Percentage,Subscribe and Save Coupon: Percentage,Month
0,com,https://m.media-amazon.com/images/I/41fcUNG75t...,"Nature's Bounty Biotin, Supports Healthy Hair,...",197.0,232.0,408.0,347.0,30000.0,4.6,92760.0,...,68.0,NaN,no,no,yes,yes,-,-,5 %,September
1,com,https://m.media-amazon.com/images/I/41wBCNhPnP...,"Nature Made Vitamin B12 1000 mcg, Dietary Supp...",294.0,253.0,784.0,308.0,1000.0,4.7,30018.0,...,45.0,NaN,no,no,yes,yes,-,-,-,September
2,com,https://m.media-amazon.com/images/I/41b8gjyBQN...,Nature Made Super B Complex with Vitamin C and...,373.0,1295.0,3096.0,157.0,100.0,4.7,24778.0,...,182.0,NaN,no,no,no,no,-,-,-,September
3,com,https://m.media-amazon.com/images/I/412G1fBj2G...,Nature Made Super B Complex with Vitamin C and...,375.0,12608.0,8989.0,222.0,2000.0,4.7,479.0,...,182.0,NaN,no,no,yes,yes,-,-,5 %,September
4,com,https://m.media-amazon.com/images/I/41X0q3Os4v...,Nature Made Super B Complex with Vitamin C and...,497.0,3373.0,2541.0,298.0,10000.0,4.6,6370.0,...,59.0,NaN,no,no,yes,yes,-,-,5 %,September


In [63]:
sep_dfC = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin C/Keepa Vitamin C Sep23.csv')
sep_dfC['Month'] = 'September'

# Load additional CSV files
oct_dfC = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin C/Keepa Vitamin C Oct23.csv')
oct_dfC = oct_dfC[oct_dfC['ASIN'].isin(sep_dfC['ASIN'])]
oct_dfC['Month'] = 'October'

nov_dfC = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin C/Keepa Vitamin C Nov23.csv')
nov_dfC = nov_dfC[nov_dfC['ASIN'].isin(sep_dfC['ASIN'])]
nov_dfC['Month'] = 'November'

dec_dfC = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin C/Keepa Vitamin C Dec23.csv')
dec_dfC = dec_dfC[dec_dfC['ASIN'].isin(sep_dfC['ASIN'])]
dec_dfC['Month'] = 'December'

jan_dfC = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin C/Keepa Vitamin C Jan24.csv')
jan_dfC = jan_dfC[jan_dfC['ASIN'].isin(sep_dfC['ASIN'])]
jan_dfC['Month'] = 'January'

feb_dfC = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin C/Keepa Vitamin C Feb24.csv')
feb_dfC = feb_dfC[feb_dfC['ASIN'].isin(sep_dfC['ASIN'])]
feb_dfC['Month'] = 'February'
def parse_sales_rank(string):
    """
    'Sales Rank: Subcategory Sales Ranks'
        "# 3 | Top 0.03% | Vitamin C Supplements"
    Split this into three:
        [3, 0.03, 'Vitamin C Supplements']
        
    Sometimes input is like
        "# 3 | Top 0.03% | Vitamin C Supplements; # 13 | Top 0.13% | Candy"
    In this case, split by ; and take the string that contains 'Vitamin C Supplements'.
    """
    # select the category that contains 'Vitamin C Supplements'
    try:
        categories = string.split(';')
    except:
        # return None if the input is not a string.
        return pd.Series([None, None, None])
    Vitamin_C = [c for c in categories if c.find('Vitamin C Supplements') > -1]
    Vitamin_C = Vitamin_C[0] if len(Vitamin_C) > 0 else None
    try:
        rank = re.findall(r'#\s(\d+)\s\|', Vitamin_C.replace(',',''))
        rank = int(rank[0].strip())
    except:
        # print('rank \n', string) # print it to see what's wrong
        rank = None  # there is Calcium Mineral Supplements which is not Vitamin C
    
    # percent
    # 1. Find the string that starts with 'Top ' and ends with '%'.
    # 2. Return the number as a float.
    # 3. If the input is not a string, return None.
    try:
        percent = re.findall(r'Top\s(.*)%', Vitamin_B)
        percent = float(percent[0].strip())
    except:
        percent = None  # percent info might not exist if ranking is too high.
        # print('percent \n', string) # print it to see what's wrong
    
    # Category
    # 1. Split the string by '|' and take the last element.
    try:
        category = Vitamin_C.split('|')[-1].strip()
    except:
        # print('category \n', string)
        category = None # there is Calcium Mineral Supplements which is not Vitamin C
    return pd.Series([rank, percent, category])
final_dfC = final_dfC.drop(columns_to_drop, axis=1, errors='ignore')
final_dfC = pd.concat([sep_dfC, oct_dfC, nov_dfC, dec_dfC, jan_dfC, feb_dfC], ignore_index=True)
final_dfC.head()

,Locale,Image,Title,Sales Rank: Historical,Sales Rank: Current,Sales Rank: 90 days avg.,Sales Rank: Drops last 90 days,Sales Rank: Reference,Sales Rank: Subcategory Sales Ranks,Bought in past month,...,Item: Weight (g),Hazardous Materials,Adult Product,Trade-In Eligible,Prime Eligible (Buy Box),Subscribe and Save,One Time Coupon: Absolute,One Time Coupon: Percentage,Subscribe and Save Coupon: Percentage,Month
0,com,https://m.media-amazon.com/images/I/712NwOYTSJ...,Emergen-C Immune+ Triple Action Immune Support...,381.0,548.0,438.0,278.0,Health & Household,# 3 | Top 0.03% | Vitamin C Supplements,10000.0,...,272.0,NaN,no,no,yes,yes,-,-,5 %,September
1,com,https://m.media-amazon.com/images/I/8178DSEUDd...,"Emergen-C 1000mg Vitamin C Powder, with Antiox...",432.0,160.0,126.0,274.0,Health & Household,# 1 | Top 0.01% | Vitamin C Supplements,5000.0,...,281.0,NaN,no,no,yes,yes,-,-,5 %,September
2,com,https://m.media-amazon.com/images/I/51b5geA8Kk...,Emergen-C 1000mg Vitamin C Powder for Daily Im...,563.0,928.0,869.0,300.0,Health & Household,# 5 | Top 0.06% | Vitamin C Supplements,10000.0,...,726.0,NaN,no,no,yes,yes,-,-,5 %,September
3,com,https://m.media-amazon.com/images/I/71wZctP0ow...,"Nature Made Chewable Vitamin C 500 mg, Dietary...",639.0,3011.0,2882.0,289.0,Health & Household,# 29 | Top 0.26% | Vitamin C Supplements,7000.0,...,159.0,NaN,no,no,yes,yes,-,-,5 %,September
4,com,https://m.media-amazon.com/images/I/71fHpn+Jnu...,Nature Made Extra Strength Dosage Chewable Vit...,639.0,1163.0,926.0,295.0,Health & Household,# 6 | Top 0.04% | Vitamin C Supplements,20000.0,...,231.0,NaN,no,no,yes,no,-,-,-,September


In [64]:
sep_dfD = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin D/Keepa Vitamin D Sep23.csv')
sep_dfD['Month'] = 'September'

# Load additional CSV files
oct_dfD = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin D/Keepa Vitamin D Oct23.csv')
oct_dfD = oct_dfD[oct_dfD['ASIN'].isin(sep_dfD['ASIN'])]
oct_dfD['Month'] = 'October'

nov_dfD = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin D/Keepa Vitamin D Nov23.csv')
nov_dfD = nov_dfD[nov_dfD['ASIN'].isin(sep_dfD['ASIN'])]
nov_dfD['Month'] = 'November'

dec_dfD = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin D/Keepa Vitamin D Dec23.csv')
dec_dfD = dec_dfD[dec_dfD['ASIN'].isin(sep_dfD['ASIN'])]
dec_dfD['Month'] = 'December'

jan_dfD = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin D/Keepa Vitamin D Jan24.csv')
jan_dfD = jan_dfD[jan_dfD['ASIN'].isin(sep_dfD['ASIN'])]
jan_dfD['Month'] = 'January'

feb_dfD = pd.read_csv('/Users/fareedkhan/Documents/Thesis/Vitamin D/Keepa Vitamin D Feb24.csv')
feb_dfD = feb_dfD[feb_dfD['ASIN'].isin(sep_dfD['ASIN'])]
feb_dfD['Month'] = 'February'
def parse_sales_rank(string):
    """
    'Sales Rank: Subcategory Sales Ranks'
        "# 3 | Top 0.03% | Vitamin D Supplements"
    Split this into three:
        [3, 0.03, 'Vitamin D Supplements']
        
    Sometimes input is like
        "# 3 | Top 0.03% | Vitamin D Supplements; # 13 | Top 0.13% | Candy"
    In this case, split by ; and take the string that contains 'Vitamin D Supplements'.
    """
    # select the category that contains 'Vitamin C Supplements'
    try:
        categories = string.split(';')
    except:
        # return None if the input is not a string.
        return pd.Series([None, None, None])
    Vitamin_D = [d for d in categories if d.find('Vitamin D Supplements') > -1]
    Vitamin_D = Vitamin_D[0] if len(Vitamin_D) > 0 else None
    try:
        rank = re.findall(r'#\s(\d+)\s\|', Vitamin_D.replace(',',''))
        rank = int(rank[0].strip())
    except:
        # print('rank \n', string) # print it to see what's wrong
        rank = None  # there is Calcium Mineral Supplements which is not Vitamin C
    
    # percent
    # 1. Find the string that starts with 'Top ' and ends with '%'.
    # 2. Return the number as a float.
    # 3. If the input is not a string, return None.
    try:
        percent = re.findall(r'Top\s(.*)%', Vitamin_D)
        percent = float(percent[0].strip())
    except:
        percent = None  # percent info might not exist if ranking is too high.
        # print('percent \n', string) # print it to see what's wrong
    
    # Category
    # 1. Split the string by '|' and take the last element.
    try:
        category = Vitamin_D.split('|')[-1].strip()
    except:
        # print('category \n', string)
        category = None # there is Calcium Mineral Supplements which is not Vitamin C
    return pd.Series([rank, percent, category])
final_dfD = final_dfD.drop(columns_to_drop, axis=1, errors='ignore')
final_dfD = pd.concat([sep_dfD, oct_dfD, nov_dfD, dec_dfD, jan_dfD, feb_dfD], ignore_index=True)
final_dfD.head()

,Locale,Image,Title,Sales Rank: Historical,Sales Rank: Current,Sales Rank: 90 days avg.,Sales Rank: Drops last 90 days,Sales Rank: Reference,Sales Rank: Subcategory Sales Ranks,Bought in past month,...,Item: Weight (g),Hazardous Materials,Adult Product,Trade-In Eligible,Prime Eligible (Buy Box),Subscribe and Save,One Time Coupon: Absolute,One Time Coupon: Percentage,Subscribe and Save Coupon: Percentage,Month
0,com,https://m.media-amazon.com/images/I/41g1x-oV9n...,NatureWise Vitamin D3 5000iu (125 mcg) 1 Year ...,36.0,16.0,23.0,234.0,Health & Household,# 3 | Climate Pledge Friendly: Health and Hous...,100000.0,...,45.0,NaN,no,no,yes,yes,-,-,5 %,September
1,com,https://m.media-amazon.com/images/I/41Y0vfI0wm...,Sports Research Vitamin D3 + K2 with 5000iu of...,105.0,118.0,240.0,243.0,Health & Household,# 2 | Top 0.03% | Vitamin D Supplements,7000.0,...,82.0,NaN,no,no,no,no,-,-,-,September
2,com,https://m.media-amazon.com/images/I/516z5YpnLf...,Sports Research Vitamin D3 5000 IU with Coconu...,238.0,874.0,1268.0,342.0,Health & Household,# 15 | Top 0.23% | Vitamin D Supplements,10000.0,...,18.0,NaN,no,no,yes,yes,-,-,10 %,September
3,com,https://m.media-amazon.com/images/I/81uSXouLLk...,Bronson Vitamin K2 (MK7) with D3 Supplement Bo...,322.0,222.0,260.0,273.0,Health & Household,# 4 | Top 0.05% | Vitamin D Supplements,5000.0,...,82.0,NaN,no,no,yes,yes,-,-,5 %,September
4,com,https://m.media-amazon.com/images/I/71JO8SJ5mB...,Nature Made Vitamin D3 1000 IU (25 mcg) Softge...,323.0,207.0,277.0,263.0,Health & Household,# 3 | Top 0.03% | Vitamin D Supplements,30000.0,...,105.0,NaN,no,no,yes,yes,-,-,5 %,September


In [66]:
import pandas as pd

# Assuming final_dfD, final_dfC, final_dfA, and final_dfB are already defined and prepared

# Concatenate the DataFrames vertically
final_combined_df = pd.concat([final_dfD, final_dfC, final_dfA, final_dfB], ignore_index=True)
 
# Count Variation ASIN
def count_variation_asin(string):
    """
    If there is no variation, the column value is NA. Then, return 1 (itself)
    """
    try:
        return len(string.split(',')) + 1  # including itself
    except:
        return 1  # itself
final_combined_df['Variation ASIN Count'] = final_combined_df['Variation ASINs'].apply(count_variation_asin)
# At this point, final_combined_df contains all the data from the four vitamin DataFrames combined
# You might want to save this final DataFrame to a new CSV for further analysis or sharing
output_file_path = '/Users/fareedkhan/Documents/Thesis/final_combined_vitamins_dataset.csv'
final_combined_df.to_csv(output_file_path, index=False)

# Output the path for confirmation
print("Final combined dataset saved to:", output_file_path)


Final combined dataset saved to: /Users/fareedkhan/Documents/Thesis/final_combined_vitamins_dataset.csv


In [67]:
final_combined_df.head()

,Locale,Image,Title,Sales Rank: Historical,Sales Rank: Current,Sales Rank: 90 days avg.,Sales Rank: Drops last 90 days,Sales Rank: Reference,Sales Rank: Subcategory Sales Ranks,Bought in past month,...,Trade-In Eligible,Prime Eligible (Buy Box),Subscribe and Save,One Time Coupon: Absolute,One Time Coupon: Percentage,Subscribe and Save Coupon: Percentage,Month,Absolute Rank,Category,Variation ASIN Count
0,com,https://m.media-amazon.com/images/I/41g1x-oV9n...,NatureWise Vitamin D3 5000iu (125 mcg) 1 Year ...,36.0,16.0,23.0,234.0,Health & Household,# 3 | Climate Pledge Friendly: Health and Hous...,100000.0,...,no,yes,yes,-,-,5 %,September,NaN,NaN,17
1,com,https://m.media-amazon.com/images/I/41Y0vfI0wm...,Sports Research Vitamin D3 + K2 with 5000iu of...,105.0,118.0,240.0,243.0,Health & Household,# 2 | Top 0.03% | Vitamin D Supplements,7000.0,...,no,no,no,-,-,-,September,NaN,NaN,4
2,com,https://m.media-amazon.com/images/I/516z5YpnLf...,Sports Research Vitamin D3 5000 IU with Coconu...,238.0,874.0,1268.0,342.0,Health & Household,# 15 | Top 0.23% | Vitamin D Supplements,10000.0,...,no,yes,yes,-,-,10 %,September,NaN,NaN,3
3,com,https://m.media-amazon.com/images/I/81uSXouLLk...,Bronson Vitamin K2 (MK7) with D3 Supplement Bo...,322.0,222.0,260.0,273.0,Health & Household,# 4 | Top 0.05% | Vitamin D Supplements,5000.0,...,no,yes,yes,-,-,5 %,September,NaN,NaN,5
4,com,https://m.media-amazon.com/images/I/71JO8SJ5mB...,Nature Made Vitamin D3 1000 IU (25 mcg) Softge...,323.0,207.0,277.0,263.0,Health & Household,# 3 | Top 0.03% | Vitamin D Supplements,30000.0,...,no,yes,yes,-,-,5 %,September,NaN,NaN,8
